<a href="https://colab.research.google.com/github/mafaldasalomao/multilabel_classification_pocus/blob/main/multilabel_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import suppporting libraries
import tarfile
import urllib.request as urllib2
import os
import cv2
from os import listdir
from os.path import isfile, join
import re
#Import deep learning libraries
import cv2
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import tqdm as tqdm
import tensorflow_hub as hub

#Import data analytics libraries
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import scipy.io
import pandas as pd
import seaborn as sns
#Import image visualization libraries
from PIL import *
from PIL import ImageFile
from PIL import Image
#System settings
ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['WANDB_CONSOLE'] = 'off'
#Coloring for print outputs
class color:
   RED = '\033[91m'
   BOLD = '\033[1m'
   END = '\033[0m'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp  /content/drive/MyDrive/datasets/convex.zip /content/

In [ ]:
!unzip convex.zip -d /content/temp

Archive:  convex.zip
   creating: /content/temp/convex/
  inflating: /content/temp/convex/Cov-Atlas+(44).gif  
  inflating: /content/temp/convex/Cov-Atlas+(45).gif  
  inflating: /content/temp/convex/Cov-Atlas-+(43).gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+1.gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+2.gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+3.gif  
  inflating: /content/temp/convex/Cov-Atlas-Day+4.gif  
  inflating: /content/temp/convex/Cov-Atlas-suspectedCovid.mp4  
  inflating: /content/temp/convex/Cov-clarius.gif  
  inflating: /content/temp/convex/Cov-clarius3.gif  
  inflating: /content/temp/convex/Cov-grep-7453.mp4  
  inflating: /content/temp/convex/Cov-grep-7505.mp4  
  inflating: /content/temp/convex/Cov-grep-7507.mp4  
  inflating: /content/temp/convex/Cov-grep-7510.mp4  
  inflating: /content/temp/convex/Cov-grep-7511.mp4  
  inflating: /content/temp/convex/Cov-grep-7525.mp4  
  inflating: /content/temp/convex/Cov-grep-7543.mp4  
  infl

In [ ]:
path='/content/temp/convex'
annotation_path = "dataset_metadata.csv"
videos_list = os.listdir(path)

In [ ]:
columns = ['Filename', 'Effusion', 'Consolidations', 'B-lines', 'A-lines', 'Pleural_line_irregularities', 'Air_bronchogram', 'Label']
base = pd.read_csv(annotation_path, usecols=columns)

In [ ]:
base = base[base['Effusion'] != 'n/A']
base = base[base['Consolidations'] != 'n/A']
base = base[base['B-lines'] != 'n/A']
base = base[base['A-lines']  != 'n/A']
base = base[base['Pleural_line_irregularities']  != 'n/A']
base = base[base['Air_bronchogram']  != 'n/A']
base = base[base['Label']  != 'n/A']
#base = base[base['Label']  != 'regular']
#base = base[base['Label']  != 'Regular']
base['Label']= base['Label'].replace(['regular'], 'Regular')

base['Effusion'] = pd.to_numeric(base['Effusion'])
base.head()

,Filename,Label,Effusion,Consolidations,B-lines,A-lines,Pleural_line_irregularities,Air_bronchogram
0,Cov-Butterfly-COVID Lung 2,COVID-19,1.0,0.0,1.0,0.0,1.0,0.0
1,Cov-Butterfly-Skip Lesion,COVID-19,0.0,1.0,1.0,1.0,1.0,0.0
2,Cov-Butterfly-Confluent B lines,COVID-19,0.0,0.0,1.0,0.0,0.0,0.0
3,Reg-Butterfly-Normal Lung A lines,Regular,0.0,0.0,0.0,1.0,0.0,0.0
4,Reg-Butterfly-Normal Lung_Example 2,Regular,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
base  = base.dropna(axis = 0)
base.shape

(346, 8)

In [ ]:
metadata = base;
metadata['Filename']

0               Cov-Butterfly-COVID Lung 2
1                Cov-Butterfly-Skip Lesion
2          Cov-Butterfly-Confluent B lines
3        Reg-Butterfly-Normal Lung A lines
4      Reg-Butterfly-Normal Lung_Example 2
                      ...                 
344                Cov-B_ConvexProb_score1
345               Cov-C_Convex_Prob_score2
346               Cov-D_Convex_Prob_score3
347                Cov-C_ConvexProb_score3
348                    Cov-MSU-SkipLesions
Name: Filename, Length: 346, dtype: object

In [ ]:
#"Cov-Butterfly-COVID Lung 2" in metadata['Filename']
teste = metadata.loc[metadata.Filename == "Reg-Atlas-lungcurtain"]
len(teste)

1

In [ ]:
effusion_list = []
consolid_list = []
aline_list = []
bline_list = []
pli_list = []
airb_list = []
filenames = []
y_labels = []
num_imgs_per_video = 10
#os.mkdir("/content/dataset")

In [ ]:
IMG_SIZE = 224 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model
num_imgs_per_video=10

In [ ]:

feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [ ]:
feature_extractor_layer.trainable = True

In [ ]:
def plot_curves(history):
    plt.figure(figsize=(5, 3))
    epochs = range(1, len(history.history['loss'])+1)
    plt.plot(epochs, history.history['loss'], label="Train loss")
    plt.plot(epochs, history.history['val_loss'], label="Test Loss")
    plt.title("Training Loss Curve")
    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend();
    
    plt.figure(figsize=(5, 3))
    epochs = range(1, len(history.history['accuracy'])+1)
    plt.plot(epochs, history.history['accuracy'], label="Train Acc")
    plt.plot(epochs, history.history['val_accuracy'], label="Test Acc")
    plt.title("Training Accuracy Curve")
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend();

## Data Augmentation


In [ ]:
!pip install -U git+https://github.com/albumentations-team/albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-505ow_a_
  Running command git clone -q https://github.com/albumentations-team/albumentations /tmp/pip-req-build-505ow_a_


In [ ]:
from albumentations import HorizontalFlip, VerticalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout

In [ ]:
def criar_diretorio(caminho):
  if not os.path.exists(caminho):
    os.makedirs(caminho)

In [ ]:
from scipy.stats.distributions import alpha
def create_dataset_aug(img_altura=224, img_largura=224, augmentation=True):
    X_train, y_train = [], []
    for filename in videos_list:
        count=0
        #print(filename)
        vid_cap = cv2.VideoCapture(path+'/'+filename)
        total_frame_count= int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = total_frame_count/num_imgs_per_video
        success, image = vid_cap.read()
        metadata_cap = metadata.loc[metadata.Filename == filename.split('.')[0]]
        if len(metadata_cap) >0:
            while success:
                #print("Reading new frame...", success)
                if ((count+1) % int(step) == 0):
                    #print("Writing new frame...")
                    out = '/content/dataset/{}_{}.png'.format(filename.split('.')[0], count)
                    cv2.imwrite(out, image)
                    img = tf.keras.utils.load_img(out, target_size = (img_largura, img_altura, 3))
                    img = tf.keras.utils.img_to_array(img)
                    img= img/255.
                    #img = img.transpose(2,0,1)
                    X_train.append(img)
                    img_labels = [metadata['A-lines'][metadata_cap.index[0]],
                                    metadata['B-lines'][metadata_cap.index[0]],
                                    metadata['Effusion'][metadata_cap.index[0]],
                                    metadata['Consolidations'][metadata_cap.index[0]],
                                    metadata['Pleural_line_irregularities'][metadata_cap.index[0]]]
                                     #metadata['Air_bronchogram'][metadata_cap.index[0]]])
                    y_train.append(img_labels)
                    if augmentation:
                      aug = HorizontalFlip(p = 1.0)
                      augmentation = aug(image = img)
                      x1 = augmentation['image']
                      X_train.append(x1)
                      y_train.append(img_labels)

                      aug = OpticalDistortion(p = 1.0, distort_limit=2, shift_limit=0.5)
                      augmentation = aug(image = img)
                      x2 = augmentation['image']
                      X_train.append(x2)
                      y_train.append(img_labels)

                      # aug = ElasticTransform(p = 1.0, alpha=120, sigma=120*0.05, alpha_affine=120*0.03)
                      # augmentation = aug(image = img)
                      # x3 = augmentation['image']
                      # X_train.append(x3)
                      # y_train.append(img_labels)

                      # aug = GridDistortion(p = 1.0)
                      # augmentation = aug(image = img)
                      # x4 = augmentation['image']
                      # X_train.append(x4)
                      # y_train.append(img_labels)

                success, image = vid_cap.read()
                count += 1
    X = np.array(X_train)
    y = np.array(y_train)
    return X, y

In [ ]:
def create_mydataset_all():
    X_train, y_train = [], []
    for filename in videos_list:
        count=0
        #print(filename)
        vid_cap = cv2.VideoCapture(path+'/'+filename)
        total_frame_count= int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
        step = total_frame_count/num_imgs_per_video
        success, image = vid_cap.read()
        metadata_cap = metadata.loc[metadata.Filename == filename.split('.')[0]]
        if len(metadata_cap) >0:
            while success:
                #print("Reading new frame...", success)
                if ((count+1) % int(step) == 0):
                    #print("Writing new frame...")
                    out = '/content/dataset/{}_{}.png'.format(filename.split('.')[0], count)
                    cv2.imwrite(out, image)
                    img = tf.keras.utils.load_img(out, target_size = (224, 224, 3))
                    img = tf.keras.utils.img_to_array(img)
                    img= img/255.
                    #img = img.transpose(2,0,1)
                    X_train.append(img)
                    y_train.append([metadata['A-lines'][metadata_cap.index[0]],
                                    metadata['B-lines'][metadata_cap.index[0]],
                                    metadata['Effusion'][metadata_cap.index[0]],
                                    metadata['Consolidations'][metadata_cap.index[0]],
                                    metadata['Pleural_line_irregularities'][metadata_cap.index[0]]])
                                     #metadata['Air_bronchogram'][metadata_cap.index[0]]])
                success, image = vid_cap.read()
                count += 1
    X = np.array(X_train)
    y = np.array(y_train)
    return X, y

In [ ]:
#X, y = create_dataset_aug()
X, y = create_mydataset_all()
#((1225, 224, 224, 3), (1225, 5))
num_folds=5
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.2, shuffle=True)
X_train.shape, y_train.shape

((1225, 224, 224, 3), (1225, 5))

In [ ]:
del X
del y
from keras import initializers
tf.random.set_seed(42)
N_LABELS = 5
EPOCHS = 20
batch_size = 32
LR = 0.00001
acc_per_fold = []
loss_per_fold = []

In [ ]:
reconstructed_model = keras.models.load_model("my_h5_model.h5")

* 1024->5 = 65%         -
* 1024->10->5 - 2x74%
* 1024->10->5   - 50%


In [ ]:
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X_train, y_train):

  # Define the model architecture
  initializer = tf.keras.initializers.Identity()
  model_all_0 = tf.keras.Sequential([
      feature_extractor_layer,
      Dense(1024, name='hidden_layer_1'),
      Dense(10, name='hidden_layer_2'),
      Dense(N_LABELS, activation='sigmoid', name='output')
  ])
  # Compile the model
  model_all_0.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])



  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model_all_0.fit(X_train[train], y_train[train],
              batch_size=batch_size,
              epochs=EPOCHS,
              validation_data=(X_test, y_test))

  # Generate generalization metrics
  scores = model_all_0.evaluate(X_train[test], y_train[test], verbose=0)
  print(f'Score for fold {fold_no}: {model_all_0.metrics_names[0]} of {scores[0]}; {model_all_0.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/20
31/31 [==============================] - 11s 256ms/step - loss: 0.5552 - accuracy: 0.3214 - val_loss: 0.3185 - val_accuracy: 0.4495
Epoch 2/20
31/31 [==============================] - 7s 232ms/step - loss: 0.2385 - accuracy: 0.4837 - val_loss: 0.1952 - val_accuracy: 0.5244
Epoch 3/20
31/31 [==============================] - 7s 234ms/step - loss: 0.1660 - accuracy: 0.5051 - val_loss: 0.1559 - val_accuracy: 0.5375
Epoch 4/20
31/31 [==============================] - 7s 238ms/step - loss: 0.1386 - accuracy: 0.5194 - val_loss: 0.1375 - val_accuracy: 0.5472
Epoch 5/20
31/31 [==============================] - 7s 234ms/step - loss: 0.1247 - accuracy: 0.5245 - val_loss: 0.1255 - val_accuracy: 0.5472
Epoch 6/20
31/31 [==============================] - 7s 231ms/step - loss: 0.1142 - accuracy: 0.5255 - val_loss: 0.1180 - val_accuracy: 0.5440
Epoch 7/20
31/31 [==============================] 

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.639514684677124 - Accuracy: 53.87755036354065%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.1047232449054718 - Accuracy: 36.73469424247742%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.08114355802536011 - Accuracy: 53.46938967704773%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.10333313047885895 - Accuracy: 50.61224699020386%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.0776873528957367 - Accuracy: 45.30612230300903%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.07579221576452255 - Accuracy: 44.08163130283356%
----------------------------------------------------------------------

In [ ]:
# Save the model
model_all_0.save('my_model_0_1(88-90).h5')

In [ ]:
N_LABELS = 5
EPOCHS = 20
LR = 0.0001
#>1024->512->5 = 68, 72
#>1024->256->5 = 63, 73
#>1024->128->5 = 63, 67
#>1024->64->5 = ----
#>1024->32->5 = 63, 67
#>1024->512->32->5 = 55, 56
#>1024>512->16->5  = 54, 56
#>1024>512->8->5  = 73, 72
model_all = tf.keras.Sequential([
    feature_extractor_layer,
    Dense(1024, activation='relu', name='hidden_layer_1'),
    Dropout(0.2),
    Dense(20, activation='relu', name='hidden_layers_2'),
    Dense(10, activation='relu', name='hidden_layers_3'),
    Dense(N_LABELS, activation='sigmoid', name='output')
])
model_all.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
  loss='binary_crossentropy',
  metrics=['accuracy'])
history_all = model_all.fit(X_train, y_train,
  epochs=EPOCHS,
  batch_size=64,
  validation_data=(X_test, y_test))

In [ ]:
# Save the model
model_all.save('my_model.h5')

In [ ]:
feature_extractor_url_resnet = "https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5"
feature_extractor_layer_resnet = hub.KerasLayer(feature_extractor_url_resnet,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))

In [ ]:
print("Thickness of the model:", len(feature_extractor_layer_resnet.weights))
#list(feature_extractor_layer_resnet.layers)

In [ ]:
feature_extractor_layer_resnet.trainable = True

In [ ]:
N_LABELS = 5
EPOCHS = 40
LR = 0.0001
batch_size = 32
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X_train, y_train):

  # Define the model architecture
  initializer = tf.keras.initializers.Identity()
  model_all_0 = tf.keras.Sequential([
      feature_extractor_layer_resnet,
      Dense(2048, activation='relu', name='hidden_layer_1'),
      Dense(526, activation='relu', name='hidden_layer_2'),
      #Dense(10, activation='relu', name='hidden_layers_3'),
      Dense(N_LABELS, activation='sigmoid', name='output')
  ])
  # Compile the model
  model_all_0.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss='binary_crossentropy',
    metrics=['accuracy'])



  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model_all_0.fit(X_train[train], y_train[train],
              batch_size=batch_size,
              epochs=EPOCHS,
              validation_data=(X_test, y_test))

  # Generate generalization metrics
  scores = model_all_0.evaluate(X_train[test], y_train[test], verbose=0)
  print(f'Score for fold {fold_no}: {model_all_0.metrics_names[0]} of {scores[0]}; {model_all_0.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/40
 1/31 [..............................] - ETA: 30:25 - loss: 0.9312 - accuracy: 0.2188

KeyboardInterrupt: ignored

In [ ]:
model_all.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 2048)              42626560  
                                                                 
 hidden_layer_1 (Dense)      (None, 542)               1110558   
                                                                 
 hidden_layers_3 (Dense)     (None, 10)                5430      
                                                                 
 output (Dense)              (None, 5)                 55        
                                                                 
Total params: 43,742,603
Trainable params: 43,644,939
Non-trainable params: 97,664
_________________________________________________________________
